# Natural Language Processing 
**Project Summary**: This project intended to explore what makes an online post popular. Here are the steps of this project.
1. Web Scraping and Download All the articles.
2. Detect Topics using LDA model
3. Try various Machine Learning Algorithms to predict the view of post 

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import urllib2
import timeit

In [2]:
online_news = pd.read_csv("/Users/wenxuanzhang/Local Doc/Documents/Work/UM/OnlineNewsPopularity.csv")
test = online_news['url'][1]
r = requests.get(test)
soup = BeautifulSoup(r.content,"html.parser")
online_news['title'] = ""
online_news["content"] =""

As there are 39644 articles,which takes more than 330 hours to collect,parse and clean, we are only going to take first 600 hundred articles to LDA analaysis 

In [63]:
# using data to estimate the standard time 
start = timeit.timeit()

for i in range(0,600):
    article = requests.get(online_news['url'][i])
    soup = BeautifulSoup(article.content,"html.parser")
    a = " ".join([x.get_text() for x in soup.find_all('p')])
    title = " ".join([x.get_text() for x in soup.find_all('p')])
    full = a + title 
    online_news['title'][i] = title
    online_news['content'][i] = full
    
end = timeit.timeit()



/Users/wenxuanzhang/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/wenxuanzhang/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Save the articles as local documents thus we could avoid web scraping the data online again and again

In [66]:
for i in range(0,600):
    filename = str(i)+'.txt'
    text_file = open(filename, "w")
    with open(filename, "w") as text_file:
        text_file.write(online_news['content'][i].encode('utf-8'))

In [79]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(art).split()]  

In [91]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]


In [106]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ladmodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [103]:
print(ldamodel.print_topics(num_topics=3, num_words=7))


[(0, u'0.029*"ap" + 0.029*"tweet" + 0.029*"twitter" + 0.018*"way" + 0.018*"medium" + 0.018*"news" + 0.018*"samsung"'), (1, u'0.009*"operation" + 0.009*"statement" + 0.009*"international" + 0.009*"la" + 0.009*"money" + 0.009*"spent" + 0.009*"clear"'), (2, u'0.009*"breakingnewscom" + 0.009*"step" + 0.009*"breaking" + 0.009*"compromising" + 0.009*"consumer" + 0.009*"account" + 0.009*"editor"')]


In [112]:
doc_lda=ladmodel[doc_term_matrix]

In [120]:
# IDEA : Using the Tom_lib model to analyze the best number of topics https://pypi.python.org/pypi/tom_lib/0.2.2
# IDEA 2: Using https://de.dariah.eu/tatom/topic_model_python.html To generate the TOPIC Distance
# IDEAR 3: Training Machine Learing model to predict the popularity of models

In [1]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')


In [2]:
import glob

In [24]:
path ='/Users/wenxuanzhang/LocalDoc/GitHub/NLP/files/*.txt'

In [134]:
# Read in file in to list 
Doc = []
files = glob.glob(path)
# iterate over the list getting each file 
for i in range(0,len(files)):
    content = open(files[i]) 
    Doc = Doc + [content.read().decode('unicode_escape').encode('ascii','ignore')]

In [139]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
from stop_words import get_stop_words
# create English stop words list
en_stop = get_stop_words('en')
from nltk.stem.porter import PorterStemmer
p_stemmer = PorterStemmer()
from gensim import corpora, models
import gensim

In [136]:
# loop through document list
texts = []
for i in Doc:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)



In [140]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)



In [143]:
# Select the number of topics in the model

[(0, u'0.025*"s" + 0.018*"read" + 0.015*"can" + 0.011*"technolog"'), (1, u'0.017*"s" + 0.006*"imag" + 0.006*"will" + 0.006*"courtesi"')]


In [145]:
ldamodel.print_topics(10)

[(0,
  u'0.025*"s" + 0.018*"read" + 0.015*"can" + 0.011*"technolog" + 0.010*"editor" + 0.009*"mashabl" + 0.009*"report" + 0.008*"new" + 0.007*"game" + 0.007*"t"'),
 (1,
  u'0.017*"s" + 0.006*"imag" + 0.006*"will" + 0.006*"courtesi" + 0.005*"can" + 0.005*"photo" + 0.005*"also" + 0.005*"one" + 0.005*"use" + 0.005*"like"')]